# PyCity Schools Analysis

- My analysis
  
---

In [1]:
import pandas as pd
from pathlib import Path

# load csv files
school_data_path = Path("Resources/schools_complete.csv")
student_data_path = Path("Resources/students_complete.csv")

# assign csv files to dataframes
school_data = pd.read_csv(school_data_path)
student_data = pd.read_csv(student_data_path)

# combine data
school_data_all = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_all.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# calculate total number of unique schools
school_count = len(school_data_all["school_name"].unique())
print(school_count)

15


In [3]:
# calculate total number of students
student_count = len(school_data_all["student_name"])
print(student_count)

39170


In [4]:
# calculate total budget
total_budget = school_data["budget"].sum()
print(total_budget)


24649428


In [5]:
# calculate average math score
average_math_score = school_data_all["math_score"].mean()
print(average_math_score)

78.98537145774827


In [6]:
# calculate average reading score
average_reading_score = school_data_all["reading_score"].mean()
print(average_reading_score)

81.87784018381414


In [7]:
# calculate the percentage of students who passed math (greater than or equal to 70)
passing_math_count = school_data_all[(school_data_all["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
print(passing_math_percentage)

74.9808526933878


In [8]:
# calculate the percentage of students who passed reading
passing_reading_count = school_data_all[(school_data_all["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
print(passing_reading_percentage)

85.80546336482001


In [9]:
# students that passed math and reading
overall_passing_count = school_data_all[
    (school_data_all["math_score"] >= 70) & (school_data_all["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = overall_passing_count / float(student_count) * 100
print(overall_passing_rate)

65.17232575950983


In [10]:
# create high-level snapshot of district's key metrics in a DataFrame
data = {'Total Schools':[school_count],'Total Students':[student_count],
        'Total Budget':[total_budget],'Average Math Score':[average_math_score],
        'Average Reading Score':[average_reading_score],'% Passing Math':[passing_math_percentage],
        '% Passing Reading':[passing_reading_percentage],'% Overall Passing':[overall_passing_rate]
        }
district_summary = pd.DataFrame(data)

# format dataframe
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [11]:
# show school names
df = school_data.groupby("school_name").count().reset_index()
df[["school_name"]].head()

,school_name
0,Bailey High School
1,Cabrera High School
2,Figueroa High School
3,Ford High School
4,Griffin High School


In [12]:
# show school types
df = school_data.groupby("type", as_index=False).count().reset_index()
df[["type"]]

,type
0,Charter
1,District


In [13]:
# total student count per school
df = school_data_all.reset_index().groupby("school_name")["student_name"].count().to_frame("Student Count")
df[["Student Count"]].head()

,Student Count
school_name,
Bailey High School,4976
Cabrera High School,1858
Figueroa High School,2949
Ford High School,2739
Griffin High School,1468


In [14]:
# total school budget
df = school_data
df["budget"] = df["budget"].map("${:,.2f}".format)
df[["school_name","budget"]].head()

,school_name,budget
0,Huang High School,"$1,910,635.00"
1,Figueroa High School,"$1,884,411.00"
2,Shelton High School,"$1,056,600.00"
3,Hernandez High School,"$3,022,020.00"
4,Griffin High School,"$917,500.00"


In [15]:
# per student budget
df = (school_data_all.groupby("school_name")["budget"].sum() /
     school_data_all.groupby("school_name")["student_name"].count()).to_frame("Per Student Budget")
df["Per Student Budget"] = df["Per Student Budget"].map("${:,.2f}".format)
df.head()

,Per Student Budget
school_name,
Bailey High School,"$3,124,928.00"
Cabrera High School,"$1,081,356.00"
Figueroa High School,"$1,884,411.00"
Ford High School,"$1,763,916.00"
Griffin High School,"$917,500.00"


In [16]:
# average math score
data = {"Average Math":[school_data_all["math_score"].mean()]}
df = pd.DataFrame(data)
df["Average Math"] = df["Average Math"].map("{:,.2f}".format)
df

,Average Math
0,78.99


In [17]:
# average reading score
data = {"Average Reading":[school_data_all["reading_score"].mean()]}
df = pd.DataFrame(data)
df["Average Reading"] = df["Average Reading"].map("{:,.2f}".format)
df

,Average Reading
0,81.88


In [18]:
# percentage of students who passed math
get_percentage = (len(student_data.loc[student_data["math_score"]>=70]) / student_count)  * 100
data = {"Passed Math":["{:,.2f}%".format(get_percentage)]}
df = pd.DataFrame(data)
df

,Passed Math
0,74.98%


In [19]:
# percentage of students who passed reading
get_percentage = (len(student_data.loc[student_data["reading_score"]>=70]) / student_count)  * 100
data = {"Passed Reading":["{:,.2f}%".format(get_percentage)]}
df = pd.DataFrame(data)
df

,Passed Reading
0,85.81%


In [20]:
# percentage of overall students who passed math and reading
get_percentage = (len(school_data_all.loc[(school_data_all["math_score"]>=70) 
& (school_data_all["reading_score"]>=70)]) / student_count)  * 100
data = {"Passed Math and Reading":["{:,.2f}%".format(get_percentage)]}
df = pd.DataFrame(data)
df

,Passed Math and Reading
0,65.17%


## Highest-Performing Schools (by % Overall Passing)

In [59]:
# highest performing schools
df = school_data_all
# data = df.reindex().groupby("school_name")[[(len((df.loc[(df["math_score"]>=70) & (df["reading_score"]>=70)])) / student_count) * 100]]
data = df.groupby("school_name").apply(lambda x: (len(x.loc[(x["math_score"]>=70) 
                                                           & (x["reading_score"]>=70)]) 
                                                           / student_count) * 100).to_frame("Percentage").reset_index()
data
# top_schools = pd.DataFrame(data)
# top_schools


,school_name,Percentage
0,Bailey High School,6.941537
1,Cabrera High School,4.332397
2,Figueroa High School,4.005617
3,Ford High School,3.796273
4,Griffin High School,3.395456
5,Hernandez High School,6.333929
6,Holden High School,0.972683
7,Huang High School,3.985193
8,Johnson High School,6.507531
9,Pena High School,2.223641


## Lowest-Performing Schools (by % Overall Passing)

In [ ]:
# lowest performing schools

bottom_schools = 
